In [1]:
#transfer learning from ImageNet on ResNet
#https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
#https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
#https://pytorch.org/tutorials/beginner/saving_loading_models.html
#https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
#https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()



import torch.utils.data
class MyDataset(torch.utils.data.Dataset):

    def __init__(self, x, y):
        super().__init__()
        self.data = torch.tensor(x, requires_grad=True)
        self.label = torch.tensor(y)

    def __len__(self):
        # Number of data point we have. Alternatively self.data.shape[0], or self.label.shape[0]
        return len(self.data)

    def __getitem__(self, idx):
        # Return the idx-th data point of the dataset
        # If we have multiple things to return (data point and label), we can return them as tuple
        data_point = self.data[idx]
        data_label = self.label[idx]
        return data_point, data_label


In [39]:
#load numpy dataset from data/schizzi with numpy
import numpy as np
import os

data = np.array([])
labels = np.array([])

count = 0
data_dir = 'data/schizzi'
#iterate over all files in data_dir
for filename in os.listdir(data_dir):
    f = os.path.join(data_dir, filename)
    
    x = np.load(f)
    x = x[:100]
    x = x.reshape(-1, 28, 28)

    y = count * np.ones(len(x))

    #repeat x on 3 channels
    x = np.stack((x,)*3, axis=1)

    if count == 0:
        data = x
        labels = y
    else:
        data = np.concatenate((data, x), axis=0)
        labels = np.concatenate((labels, y), axis=0)

    count += 1
    if count == 5:
        break


In [40]:
dataset = MyDataset(data.astype(np.float32), labels)

In [41]:
#calculate mean and std
DATA_MEANS = (dataset.data / 255.0).mean(axis=(0,1,2))
DATA_STD = (dataset.data / 255.0).std(axis=(0,1,2))

In [42]:
import random

# Function for setting the seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [43]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(DATA_MEANS, DATA_STD)
])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #use GPU if available

#split dataset into train, validation and test
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)

In [44]:
print(next(iter(train_loader))[0].shape)

torch.Size([128, 3, 28, 28])


In [45]:
model_conv = torchvision.models.resnet18(weights='IMAGENET1K_V1')
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, count)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [46]:
def train_model(net, criterion, optimizer, max_epochs=50):
        
    results = None
    val_scores = []
    train_losses, train_scores = [], []
    best_val_epoch = -1
    net = net.double()
    for epoch in range(max_epochs):
        ############
        # Training #
        ############
        net.train()
        true_preds, count = 0., 0
        i = 0
        for imgs, labels in train_loader:
            print(f"Batch {i}/{len(train_loader)}")
            i += 1
            imgs = imgs.double()
            labels = labels.long()
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            preds = net(imgs)
            #softmax
            preds = torch.functional.F.softmax(preds, dim=-1)
            loss = criterion(preds, labels)
            loss.backward()
            optimizer.step()
            # Record statistics during training
            true_preds += (preds.argmax(dim=-1) == labels).sum().item()
            count += labels.shape[0]
            train_losses.append(loss.item())
        train_acc = true_preds / count
        train_scores.append(train_acc)

        ##############
        # Validation #
        ##############
        val_acc = test_model(net, val_loader)
        val_scores.append(val_acc)
        print(f"[Epoch {epoch+1:2d}] Training accuracy: {train_acc*100.0:05.2f}%, Validation accuracy: {val_acc*100.0:05.2f}%")
    
    return results


In [47]:
def test_model(net, data_loader):
    net.eval()
    true_preds, count = 0., 0
    for imgs, labels in data_loader:
        imgs = imgs.double()
        labels = labels.long()
        imgs, labels = imgs.to(device), labels.to(device)
        with torch.no_grad():
            preds = net(imgs)
            #softmax
            preds = torch.functional.F.softmax(preds, dim=-1)
            preds = preds.argmax(dim=-1)
            true_preds += (preds == labels).sum().item()
            count += labels.shape[0]
    test_acc = true_preds / count
    return test_acc 

In [48]:
return_val = train_model(model_conv, criterion, optimizer_conv, max_epochs=5)

Batch 0/4
Batch 1/4
Batch 2/4
Batch 3/4
[Epoch  1] Training accuracy: 18.25%, Validation accuracy: 22.00%
Batch 0/4
Batch 1/4
Batch 2/4
Batch 3/4
[Epoch  2] Training accuracy: 15.75%, Validation accuracy: 18.00%
Batch 0/4
Batch 1/4
Batch 2/4
Batch 3/4
[Epoch  3] Training accuracy: 17.00%, Validation accuracy: 16.00%
Batch 0/4
Batch 1/4
Batch 2/4
Batch 3/4
[Epoch  4] Training accuracy: 19.00%, Validation accuracy: 30.00%
Batch 0/4
Batch 1/4
Batch 2/4
Batch 3/4
[Epoch  5] Training accuracy: 19.75%, Validation accuracy: 28.00%


In [49]:
#test model
test_acc = test_model(model_conv.double(), test_loader)

In [53]:
#print len of test_loader
print(len(test_dataset))
print(f"Test accuracy: {test_acc}")

50
Test accuracy: 0.14
